In [ ]:
!pip install transformers
!pip install -U PyPDF2
!pip install python-docx
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=18ea6e12a998827f5c2d2d0e5d1f960e36e09247894054faee9a50f78dd55cd4
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [ ]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read documents from the directory
train_directory = '/content/drive/MyDrive/Musuem_Data/Conversational_QA.txt'
text_data = read_txt(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [ ]:
with open("/content/drive/MyDrive/Musuem_Data/Conversational_QA.txt", "w") as f:
    f.write(text_data)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()


'''def train(directory, model_output_path, train_fraction=0.8):
    # Read documents from the directory
    combined_text = read_txt(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

    # Split the text into training and validation sets
    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]

    # Save the training and validation data as text files
    with open("train.txt", "w") as f:
        f.write(train_text)
    with open("val.txt", "w") as f:
        f.write(val_text)

    # Set up the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
    model = GPT2LMHeadModel.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

    # Prepare the dataset
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Set up the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=4,
        num_train_epochs=num_train_epochs,
        save_steps= 50000,
        save_total_limit=2,
        logging_dir='./logs',
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    trainer.save_model(model_output_path)

    # Save the tokenizer
    tokenizer.save_pretrained(model_output_path)'''


'def train(directory, model_output_path, train_fraction=0.8):\n    # Read documents from the directory\n    combined_text = read_txt(directory)\n    combined_text = re.sub(r\'\n+\', \'\n\', combined_text).strip()  # Remove excess newline characters\n\n    # Split the text into training and validation sets\n    split_index = int(train_fraction * len(combined_text))\n    train_text = combined_text[:split_index]\n    val_text = combined_text[split_index:]\n\n    # Save the training and validation data as text files\n    with open("train.txt", "w") as f:\n        f.write(train_text)\n    with open("val.txt", "w") as f:\n        f.write(val_text)\n\n    # Set up the tokenizer and model\n    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl\n    model = GPT2LMHeadModel.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl\n\n    # Prepare the dataset\n    train_dataset = TextDataset(tokenizer=to

In [ ]:
train_file_path = "/content/drive/MyDrive/Musuem_Data/Conversational_QA.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/Musuem_Data/Models/Conversational_Model'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 60.0
save_steps = 50000

In [ ]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

'''def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))'''

def generate_text(model_path, sequence, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    generated_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)

    # Find the first occurrence of [Guide] and [Visitor] in the generated text
    guide_start_index = generated_text.find('[Guide]')
    visitor_start_index = generated_text.find('[Visitor]')

    # Extract the generated text between [Guide] and [Visitor]
    if guide_start_index != -1 and visitor_start_index != -1:
        generated_text = generated_text[guide_start_index + 8:visitor_start_index].strip()
    elif guide_start_index != -1:
        generated_text = generated_text[guide_start_index + 8:].strip()

    return generated_text

In [ ]:
model_path = "/content/drive/MyDrive/Musuem_Data/Models/Conversational_Model"
print("Welcome to the Louvre Museum! I'm here to provide you with information and answer any questions you may have")
end_keywords = ['no', 'thank you', 'thanks','thank you!','thanks!']

while True:
    sequence = input("User: ")
    if sequence.lower() in end_keywords:
      print("Enjoy your time exploring the incredible art at the Louvre Museum!")
      break
    else:
      max_len = 100
      answer = generate_text(model_path, sequence, max_len)
      if answer == '':
          print("Sorry, but I have no information about your question.")
      else:
          print("Guide: " + answer)
          print("Guide: Do you need anything else?")
          exit_command = input("User: ").lower()
          if exit_command in end_keywords:
              print("Enjoy your time exploring the incredible art at the Louvre Museum!")
              break

Welcome to the Louvre Museum! I'm here to provide you with information and answer any questions you may have
User: Where is this Museum located?
Guide: The Louvre Museum is located in Paris, France. It's a magnificent museum with a rich collection of art.
Guide: Do you need anything else?
User: thanks
Enjoy your time exploring the incredible art at the Louvre Museum!
